In [1]:
! pip install chess
! pip install pytorch-lightning


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from collections import OrderedDict
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from torch import nn

class EvaluationModel(pl.LightningModule):
  def __init__(self,learning_rate=1e-3,batch_size=512,layer_count=4):
    super().__init__()
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    layers = []
    for i in range(layer_count-1):
      layers.append((f"linear-{i}", nn.Linear(768, 768)))
      layers.append((f"relu-{i}", nn.ReLU()))
    layers.append((f"linear-{layer_count-1}", nn.Linear(768, 1)))
    self.seq = nn.Sequential(OrderedDict(layers))

  def forward(self, x):
    return self.seq(x)

  def training_step(self, batch, batch_idx):
    x, y = batch['bitboard'], batch['eval']
    y_hat = self(x)
    loss = F.l1_loss(y_hat, y)
    self.log("train_loss", loss)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

configs = [
           {"layer_count": 4, "batch_size": 512},
            # {"layer_count": 6, "batch_size": 1024},
           ]

checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/checkpoints/",
    filename="model-{step}",
    save_top_k=-1,  # Save all checkpoints if needed, adjust if necessary
    every_n_train_steps= 3000,  # Save checkpoint every 100 training steps (batches)
)

# 加載訓練好的 Lightning 模型
model = EvaluationModel.load_from_checkpoint("model-step=51000.ckpt", layer_count=4, batch_size=512, learning_rate=1e-3)


# 設置模型為評估模式
model.eval()

# # 保存為 PyTorch 模型
# torch.save(model.state_dict(), "evaluation.pt")

EvaluationModel(
  (seq): Sequential(
    (linear-0): Linear(in_features=768, out_features=768, bias=True)
    (relu-0): ReLU()
    (linear-1): Linear(in_features=768, out_features=768, bias=True)
    (relu-1): ReLU()
    (linear-2): Linear(in_features=768, out_features=768, bias=True)
    (relu-2): ReLU()
    (linear-3): Linear(in_features=768, out_features=1, bias=True)
  )
)

In [3]:
def evaluate_board(model, board, device):
    """
    使用模型評估當前棋盤狀態。
    :param model: PyTorch 加載的模型
    :param board: chess.Board 對象
    :return: 棋局的評估值
    """
    # 將棋盤轉換為 768 維的張量
    board_tensor = torch.zeros(768, dtype=torch.float32)  # 初始化為 768 維的全零張量
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            # 假設每個棋子都有一個唯一的編碼
            piece_type = piece.piece_type  # 棋子類型（例如，1 表示兵，6 表示王）
            color = int(piece.color)  # 棋子顏色（0 表示白色，1 表示黑色）
            idx = i * 12 + (piece_type - 1) + (6 * color)
            board_tensor[idx] = 1  # 將對應位置設為 1

    # 增加 batch 維度，並移動到設備
    board_tensor = board_tensor.unsqueeze(0).to(device)

    # 模型推論
    with torch.no_grad():
        evaluation = model(board_tensor).item()  # 返回評估值
    return evaluation

In [4]:
def alpha_beta_pruning(board, depth, alpha, beta, is_maximizing, model, device, transposition_table):
    """
    帶有 Transposition Table 的 Alpha-Beta 剪枝。
    """
    # 生成棋盤的唯一哈希值（可以用 board.fen() 或 zobrist 哈希）
    board_hash = board.fen()

    # 檢查置換表是否已有該棋盤狀態
    if board_hash in transposition_table:
        stored_depth, stored_value = transposition_table[board_hash]
        if stored_depth >= depth:
            return stored_value  # 如果存儲的深度足夠深，直接返回評估值

    if depth == 0 or board.is_game_over():
        eval_value = evaluate_board(model, board, device)
        transposition_table[board_hash] = (depth, eval_value)  # 存入置換表
        return eval_value

    legal_moves = list(board.legal_moves)

    if is_maximizing:
        max_eval = float('-inf')
        for move in legal_moves:
            board.push(move)  # 執行走法
            eval = alpha_beta_pruning(board, depth - 1, alpha, beta, False, model, device, transposition_table)
            board.pop()  # 回退走法
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta 剪枝
        transposition_table[board_hash] = (depth, max_eval)  # 存入置換表
        return max_eval
    else:
        min_eval = float('inf')
        for move in legal_moves:
            board.push(move)  # 執行走法
            eval = alpha_beta_pruning(board, depth - 1, alpha, beta, True, model, device, transposition_table)
            board.pop()  # 回退走法
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha 剪枝
        transposition_table[board_hash] = (depth, min_eval)  # 存入置換表
        return min_eval
import random

def find_best_move(board, depth, model, device, tolerance=1.0):
    """
    尋找最佳走法，結合隨機性。
    :param board: 棋盤對象
    :param depth: 搜索深度
    :param model: 模型
    :param device: 設備 (GPU 或 CPU)
    :param tolerance: 評估值的容忍範圍，用於選擇多個候選走法
    :return: 隨機選擇的最佳走法
    """
    transposition_table = {}  # 初始化置換表
    best_value = float('-inf')
    alpha = float('-inf')
    beta = float('inf')

    # 用於存儲候選走法及其評估值
    candidate_moves = []

    for move in board.legal_moves:
        board.push(move)  # 執行當前走法
        eval = alpha_beta_pruning(board, depth - 1, alpha, beta, False, model, device, transposition_table)
        board.pop()  # 回退當前走法

        # 如果該走法的評估值接近最佳值，加入候選
        if eval > best_value - tolerance:
            candidate_moves.append((move, eval))
            best_value = max(best_value, eval)

    # 如果有多個候選走法，隨機選擇一個
    if len(candidate_moves) > 1:
        chosen_move = random.choice(candidate_moves)[0]
    else:
        chosen_move = candidate_moves[0][0]  # 只有一個候選時直接返回

    return chosen_move

In [1]:
import chess
import chess.svg
from IPython.display import SVG, display, clear_output
import torch

# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 將模型移動到指定設備
model = model.to(device)

# 初始化棋盤
board = chess.Board()  # 新遊戲開始

# 設置搜索深度
depth = 4

# 交互式下棋
while not board.is_game_over():
    # 清除輸出並顯示棋盤
    # print(board)
    display(SVG(chess.svg.board(board=board, size=400)))

    # 人類玩家走棋
    print("Your turn! Enter your move in UCI format (e.g., e2e4):")
    move_input = input()

    if move_input == 'gg':  # 用戶可以輸入 'gg' 結束遊戲
        print("Game terminated by player.")
        break
    
    try:
        # 嘗試執行玩家輸入的走法
        move = chess.Move.from_uci(move_input)
        if move in board.legal_moves:
            board.push(move)
        else:
            print("Illegal move! Try again.")
            continue
    except ValueError:
        print("Invalid input format! Try again.")
        continue

    # 檢查是否結束
    if board.is_game_over():
        break

    # AI 模型走棋
    print("AI is thinking...")
    best_move = find_best_move(board, depth, model, device)
    print(f"AI plays: {best_move}")
    board.push(best_move)

# 打印最終棋盤並設置大小
clear_output(wait=True)
display(SVG(chess.svg.board(board=board, size=400)))  # 顯示最終棋盤
print("Game over!")

# 打印比賽結果
if board.is_checkmate():
    if board.turn:  # 如果輪到白方但已經將死
        print("Black wins by checkmate!")
    else:
        print("White wins by checkmate!")
elif board.is_stalemate():
    print("Draw by stalemate!")
elif board.is_insufficient_material():
    print("Draw by insufficient material!")
elif board.is_seventyfive_moves():
    print("Draw by 75-move rule!")
elif board.is_fivefold_repetition():
    print("Draw by fivefold repetition!")
else:
    print("Draw!")



NameError: name 'model' is not defined